In [ ]:
import os
import shutil

# 1. Clean up previous broken folders
folders_to_clean = ["ACE-Step-1.5-GCollab-", "acestep_installation", "checkpoints"]
print("🧹 Cleaning up old installations...")
for folder in folders_to_clean:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"   Deleted {folder}")

# 2. Install High-Speed Downloader & Audio Tools
print("🔧 Installing System Tools (aria2 & ffmpeg)...")
!apt-get install -y aria2 ffmpeg > /dev/null

print("✅ System Ready.")

In [ ]:
import os
# 1. Clone the repository
print("⬇️ Cloning ACE-Step Code...")
!git clone https://github.com/AgentHitmanFaris/ACE-Step-1.5-GCollab- aceptest_app

# 2. Enter the folder
os.chdir("aceptest_app")
print(f"📍 Working Directory: {os.getcwd()}")

# 3. Install Python Dependencies
print("📦 Installing Python Libraries (This takes ~2 mins)...")
!pip install -r requirements.txt
!pip install loguru diffusers accelerate transformers gradio modelscope soundfile einops vector-quantize-pytorch torchcodec peft pytorch-lightning huggingface_hub

# 4. CRITICAL FIX: Install nano-vllm
print("🧠 Installing nano-vllm (Critical Fix)...")
!pip install -e acestep/third_parts/nano-vllm

# 5. Fix Flash Attention compatibility
!pip uninstall -y flash_attn

print("✅ Code & Libraries Installed.")

In [ ]:
import os

# Helper to download with aria2
def download_file(url, folder, filename):
    os.makedirs(folder, exist_ok=True)
    print(f"⬇️ Downloading {filename}...")
    !aria2c -x 16 -s 16 -d {folder} -o {filename} {url} > /dev/null

# --- 1. Main Turbo Model & Configs ---
base_turbo = "https://huggingface.co/ACE-Step/Ace-Step1.5/resolve/main/acestep-v15-turbo"
turbo_dir = "checkpoints/acestep-v15-turbo"
download_file(f"{base_turbo}/model.safetensors", turbo_dir, "model.safetensors")
download_file(f"{base_turbo}/config.json", turbo_dir, "config.json")
download_file(f"{base_turbo}/configuration_acestep_v15.py", turbo_dir, "configuration_acestep_v15.py")
download_file(f"{base_turbo}/modeling_acestep_v15_turbo.py", turbo_dir, "modeling_acestep_v15_turbo.py")
download_file(f"{base_turbo}/silence_latent.pt", turbo_dir, "silence_latent.pt") # The file you were missing!

# --- 2. VAE (Audio Decoder) ---
base_vae = "https://huggingface.co/ACE-Step/Ace-Step1.5/resolve/main/vae"
vae_dir = "checkpoints/vae"
download_file(f"{base_vae}/diffusion_pytorch_model.safetensors", vae_dir, "diffusion_pytorch_model.safetensors")
download_file(f"{base_vae}/config.json", vae_dir, "config.json")

# --- 3. Text Encoder (Qwen) - Fixed Missing Folder ---
base_text = "https://huggingface.co/ACE-Step/Qwen3-Embedding-0.6B/resolve/main"
text_dir = "checkpoints/Qwen3-Embedding-0.6B"
download_file(f"{base_text}/model.safetensors", text_dir, "model.safetensors")
download_file(f"{base_text}/config.json", text_dir, "config.json")
download_file(f"{base_text}/tokenizer.json", text_dir, "tokenizer.json")
download_file(f"{base_text}/tokenizer_config.json", text_dir, "tokenizer_config.json")
download_file(f"{base_text}/vocab.json", text_dir, "vocab.json")
download_file(f"{base_text}/merges.txt", text_dir, "merges.txt")

# --- 4. Language Model (1.7B) - Optional but downloaded to prevent crashes ---
base_lm = "https://huggingface.co/ACE-Step/acestep-5Hz-lm-1.7B/resolve/main"
lm_dir = "checkpoints/acestep-5Hz-lm-1.7B"
download_file(f"{base_lm}/model.safetensors", lm_dir, "model.safetensors")
download_file(f"{base_lm}/config.json", lm_dir, "config.json")
download_file(f"{base_lm}/tokenizer.json", lm_dir, "tokenizer.json")
download_file(f"{base_lm}/tokenizer_config.json", lm_dir, "tokenizer_config.json")

print("✅ All Models Downloaded Successfully!")

In [ ]:
import os
import shutil
from huggingface_hub import snapshot_download

# 1. Target the broken folder
text_encoder_path = "/content/aceptest_app/checkpoints/Qwen3-Embedding-0.6B"

print(f"🗑️ Deleting corrupted folder: {text_encoder_path}")
if os.path.exists(text_encoder_path):
    shutil.rmtree(text_encoder_path)

# 2. Re-download using the official Hub tool (Verify integrity)
print("⬇️ Re-downloading Text Encoder (Qwen3)...")
try:
    snapshot_download(
        repo_id="ACE-Step/Qwen3-Embedding-0.6B",
        local_dir=text_encoder_path,
        local_dir_use_symlinks=False, # Force real files
        ignore_patterns=["*.git*"]    # Skip git files
    )
    print("✅ Download Success!")
    
    # 3. Verify the file actually works now
    config_file = os.path.join(text_encoder_path, "config.json")
    if os.path.exists(config_file):
        with open(config_file, "r") as f:
            content = f.read()
            if "model_type" in content:
                print("✅ Config file verified (contains 'model_type').")
            else:
                print("⚠️ WARNING: Config file looks weird. Please check internet.")
    else:
        print("❌ Config file missing after download.")
        
except Exception as e:
    print(f"❌ Download Failed: {e}")

# 4. Patch the Crash Bug (One last time to be safe)
file_path = "acestep/gradio_ui/events/generation_handlers.py"
if os.path.exists(file_path):
    with open(file_path, "r") as f:
        code = f.read()
    if 'gr.update(),              # 14' in code and 'gr.update(),              # 15' not in code:
        code = code.replace('gr.update(),              # 14', 'gr.update(),              # 14\n                gr.update(),              # 15')
        with open(file_path, "w") as f:
            f.write(code)
        print("🩹 Code patched for '15 vs 16' crash.")

# 5. Restart
print("\n🚀 Restarting App...")
!python -m acestep.acestep_v15_pipeline